# ML Models

This notebook will examine ML model generation. We will load in unified_complete.parquet which contains all 3 modes of trasnport we have explore thus far (taxi, subway, and bike). It contains a unified measure for transport, transport_total, which represents the total number of individuals in a given locationID on a given hour. 

Our dataframe is from the 1st of February 2022 to the 1st of April 2023 as we only selected months in which we had data for all three modes of transport. As we have seen from the different busyness graphs these different modes of transport experience different busyness / usage patterns so using just one mode of transport alone (say taxi) would be a poor proxy for busyness. With all three modes of transport we have a more complete picture of busyness patterns in each location ID.

For the MVP we will just be focusing on transport_total (it is our target variable), though it would be feasible to use this dataset as a basis for models for predicting individual modes of transport also. As this is not necessary, we will not be doing this for the time being. 

We will begin by loading in the dataset and creating dummy variables for hour, day, and month. For ease of use on the website we may opt to group predictions so that our predictions our for grouped time periods e.g. 3 hours rather than just one hour. 

We will explore different machine learning models, examining performance and seeing which best work with our data. 

We will be careful to avoid overfitting also. 

We wish to achieve a tradeoff between performance and resource_efficiency

## Load in the dataset and make preparations

In [74]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [75]:
initial_df = pd.read_parquet('unified_complete.parquet')
initial_df

,LocationID,datetime,bike_count,passenger_count,ridership,transport_total
725,4,2022-02-01 00:00:00,11.0,20.0,NaN,31.0
726,4,2022-02-01 01:00:00,5.0,8.0,NaN,13.0
727,4,2022-02-01 02:00:00,1.0,8.0,NaN,9.0
728,4,2022-02-01 03:00:00,2.0,1.0,NaN,3.0
729,4,2022-02-01 04:00:00,1.0,5.0,NaN,6.0
...,...,...,...,...,...,...
800273,261,2023-02-18 06:00:00,NaN,NaN,205.0,205.0
800274,261,2023-02-21 04:00:00,NaN,NaN,43.0,43.0
800275,261,2023-02-27 03:00:00,NaN,NaN,19.0,19.0
800276,261,2023-02-28 04:00:00,NaN,NaN,45.0,45.0


In [76]:
# Extract hour, day, and month as separate columns
initial_df['hour'] = initial_df['datetime'].dt.hour
initial_df['day'] = initial_df['datetime'].dt.dayofweek
initial_df['month'] = initial_df['datetime'].dt.month
initial_df['week_of_month'] = (initial_df['datetime'].dt.day - 1) // 7

df_dummies = initial_df

In [77]:
df_dummies.drop(['bike_count', 'passenger_count', 'ridership', 'datetime'], axis=1, inplace=True)

In [78]:
df_dummies

,LocationID,transport_total,hour,day,month,week_of_month
725,4,31.0,0,1,2,0
726,4,13.0,1,1,2,0
727,4,9.0,2,1,2,0
728,4,3.0,3,1,2,0
729,4,6.0,4,1,2,0
...,...,...,...,...,...,...
800273,261,205.0,6,5,2,2
800274,261,43.0,4,1,2,2
800275,261,19.0,3,0,2,3
800276,261,45.0,4,1,2,3


### Create new dataframe for ML

In [79]:
df_ml = df_dummies

In [80]:
df_ml.columns

Index(['LocationID', 'transport_total', 'hour', 'day', 'month',
       'week_of_month'],
      dtype='object')

In [81]:
df_ml

,LocationID,transport_total,hour,day,month,week_of_month
725,4,31.0,0,1,2,0
726,4,13.0,1,1,2,0
727,4,9.0,2,1,2,0
728,4,3.0,3,1,2,0
729,4,6.0,4,1,2,0
...,...,...,...,...,...,...
800273,261,205.0,6,5,2,2
800274,261,43.0,4,1,2,2
800275,261,19.0,3,0,2,3
800276,261,45.0,4,1,2,3


## Random Forests

In [82]:
zones_dict = {}


# Iterate over unique LocationIDs in df_ml
for ID in df_ml['LocationID'].unique():
    
    # Select rows with matching LocationID and drop the LocationID column
    df_location = df_ml.loc[df_ml['LocationID'] == ID].drop(['LocationID'], axis=1)
    
    # Reset the index and drop the old index column
    df_location = df_location.reset_index(drop=True)
    
    # Add the dataframe to the zones_dict
    zones_dict[ID] = df_location

print("Zones Dict")
print(zones_dict)

modelfeatures = ['hour', 'day', 'month', 'week_of_month']

# Create models dictionary
models_dict = {}

for ID in zones_dict:
    # Extract input and output data from zones_dict
    X = zones_dict[ID][modelfeatures].values
    y = zones_dict[ID]['transport_total'].values

    print("******* Zone: %s *******" % ID)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    print("Using %s rows and %s columns." % X_train.shape)

    # Create and fit random forest regressor model
    rf = RandomForestRegressor(n_estimators=100)
    rf.fit(X_train, y_train)

    print("Testing on %s rows." % y_test.shape[0])
    # Store model in dictionary
    models_dict[ID] = rf


Zones Dict
{4:        transport_total  hour  day  month  week_of_month
0                 31.0     0    1      2              0
1                 13.0     1    1      2              0
2                  9.0     2    1      2              0
3                  3.0     3    1      2              0
4                  6.0     4    1      2              0
...                ...   ...  ...    ...            ...
10166              5.0     5    6     12              3
10167              2.0     2    0      1              0
10168              4.0     1    0      1              3
10169              2.0     3    1      2              3
10170              2.0     2    0      3              0

[10171 rows x 5 columns], 12:       transport_total  hour  day  month  week_of_month
0                 3.0     7    1      2              0
1                 5.0     8    1      2              0
2                 2.0    10    1      2              0
3                 1.0    13    1      2              0
4      

Took about 1 minute to run random forests above 

Verifying that the features used are as intended (we will pick a model for a random zone and see whether this is correct)

In [83]:
zones_dict[4][modelfeatures]

,hour,day,month,week_of_month
0,0,1,2,0
1,1,1,2,0
2,2,1,2,0
3,3,1,2,0
4,4,1,2,0
...,...,...,...,...
10166,5,6,12,3
10167,2,0,1,0
10168,1,0,1,3
10169,3,1,2,3


In [84]:
# # Iterate over station IDs in models_dict
# results_dict = {}


# for ID in models_dict:
    
#     # Extract input and output data
#     X_test = zones_dict[ID][modelfeatures].values
#     y_test = zones_dict[ID]['transport_total']
    
#     # Make predictions on test set
#     y_pred = models_dict[ID].predict(X_test)
    
#     # Calculate absolute error and percentage difference
#     abs_error = abs(y_test - y_pred)
#     pct_error = abs_error / y_test * 100
    
#     # Store results in dictionary
#     results_dict[ID] = {'y_test': y_test, 'y_pred': y_pred, 'abs_error': abs_error, 'pct_error': pct_error}
    
#     # Print results for each station
#     print(f"***** Zone {ID} *****")
#     actual_vs_predicted = pd.concat([pd.DataFrame(y_test.values, columns=['Actual']), pd.DataFrame(y_pred, columns=['Predicted']), pd.DataFrame(pct_error.values, columns=['% Difference'])], axis=1)
#     print(actual_vs_predicted)


In [85]:
results_dict = {}

for ID in models_dict:

    # Extract input and output data
    X_test = zones_dict[ID][modelfeatures].values
    y_test = zones_dict[ID]['transport_total']

    # Make predictions on test set
    y_pred = models_dict[ID].predict(X_test)

    # Convert predictions to whole positive numbers
    y_pred = np.round(y_pred).astype(int)
    y_pred = np.abs(y_pred)

    # Calculate absolute error and percentage difference
    abs_error = np.abs(y_test - y_pred)
    pct_error = abs_error / y_test * 100

    # Store results in dictionary
    results_dict[ID] = {'y_test': y_test, 'y_pred': y_pred, 'abs_error': abs_error, 'pct_error': pct_error}

    # Print results for each station
    print(f"***** Zone {ID} *****")
    actual_vs_predicted = pd.concat([pd.DataFrame(y_test.values, columns=['Actual']), pd.DataFrame(y_pred, columns=['Predicted']), pd.DataFrame(pct_error.values, columns=['% Difference'])], axis=1)
    print(actual_vs_predicted)


***** Zone 4 *****
       Actual  Predicted  % Difference
0        31.0         35     12.903226
1        13.0         15     15.384615
2         9.0         10     11.111111
3         3.0          4     33.333333
4         6.0          6      0.000000
...       ...        ...           ...
10166     5.0          8     60.000000
10167     2.0          4    100.000000
10168     4.0         10    150.000000
10169     2.0          6    200.000000
10170     2.0         11    450.000000

[10171 rows x 3 columns]
***** Zone 12 *****
      Actual  Predicted  % Difference
0        3.0          3      0.000000
1        5.0          9     80.000000
2        2.0          4    100.000000
3        1.0          2    100.000000
4        4.0          4      0.000000
...      ...        ...           ...
8279     3.0          4     33.333333
8280     1.0          1      0.000000
8281     1.0          2    100.000000
8282     1.0          2    100.000000
8283     1.0          4    300.000000

[8284 rows

In [86]:
results_dict = {}

for ID in models_dict:
    # Extract input and output data
    X_test = zones_dict[ID][modelfeatures].values
    y_test = zones_dict[ID]['transport_total']
    
    # Make predictions on test set
    y_pred = models_dict[ID].predict(X_test)
    
    # Round the predictions to the nearest integer
    y_pred_rounded = np.round(y_pred)
    
    # Calculate absolute error and percentage difference
    abs_error = abs(y_test - y_pred_rounded)
    pct_error = abs_error / y_test * 100
    
    # Store results in dictionary
    results_dict[ID] = {'y_test': y_test, 'y_pred': y_pred_rounded, 'abs_error': abs_error, 'pct_error': pct_error}
    
    # Print results for each station
    print(f"***** Zone {ID} *****")
    actual_vs_predicted = pd.concat([pd.DataFrame(y_test.values, columns=['Actual']), pd.DataFrame(y_pred_rounded, columns=['Predicted']), pd.DataFrame(pct_error.values, columns=['% Difference'])], axis=1)
    print(actual_vs_predicted)

***** Zone 4 *****
       Actual  Predicted  % Difference
0        31.0       35.0     12.903226
1        13.0       15.0     15.384615
2         9.0       10.0     11.111111
3         3.0        4.0     33.333333
4         6.0        6.0      0.000000
...       ...        ...           ...
10166     5.0        8.0     60.000000
10167     2.0        4.0    100.000000
10168     4.0       10.0    150.000000
10169     2.0        6.0    200.000000
10170     2.0       11.0    450.000000

[10171 rows x 3 columns]
***** Zone 12 *****
      Actual  Predicted  % Difference
0        3.0        3.0      0.000000
1        5.0        9.0     80.000000
2        2.0        4.0    100.000000
3        1.0        2.0    100.000000
4        4.0        4.0      0.000000
...      ...        ...           ...
8279     3.0        4.0     33.333333
8280     1.0        1.0      0.000000
8281     1.0        2.0    100.000000
8282     1.0        2.0    100.000000
8283     1.0        4.0    300.000000

[8284 rows

In [87]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Initialize dictionaries and variables to store metrics
mae_dict = {}
mse_dict = {}
rmse_dict = {}
r2_dict = {}
adj_r2_dict = {}
mae_total = 0
mse_total = 0
rmse_total = 0
r2_total = 0
adj_r2_total = 0
num_zones = len(results_dict)

# Iterate over station IDs in results_dict
for ID in results_dict:
    
    # Extract test labels and predictions
    y_test = results_dict[ID]['y_test']
    y_pred = results_dict[ID]['y_pred']
    
    # Calculate MAE and store in dictionary
    mae = mean_absolute_error(y_test, y_pred)
    mae_dict[ID] = mae
    mae_total += mae
    
    # Calculate MSE and store in dictionary
    mse = mean_squared_error(y_test, y_pred)
    mse_dict[ID] = mse
    mse_total += mse
    
    # Calculate RMSE and store in dictionary
    rmse = np.sqrt(mse)
    rmse_dict[ID] = rmse
    rmse_total += rmse
    
    # Calculate R2 and store in dictionary
    r2 = r2_score(y_test, y_pred)
    r2_dict[ID] = r2
    r2_total += r2
    
    # Calculate adjusted R2 and store in dictionary
    n = len(y_test)
    p = len(modelfeatures)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    adj_r2_dict[ID] = adj_r2
    adj_r2_total += adj_r2
    
    # # Print results for each station
    # print(f"***** Station {ID} *****")
    # print(f"MAE: {mae:.2f}")
    # print(f"MSE: {mse:.2f}")
    # print(f"RMSE: {rmse:.2f}")
    # print(f"R2: {r2:.2f}")
    # print(f"Adjusted R2: {adj_r2:.2f}")
    
# Calculate overall averages
mae_avg = mae_total / num_zones
mse_avg = mse_total / num_zones
rmse_avg = rmse_total / num_zones
r2_avg = r2_total / num_zones
adj_r2_avg = adj_r2_total / num_zones

# Print overall averages
print("***** Overall Averages *****")
print(f"MAE: {mae_avg:.2f}")
print(f"MSE: {mse_avg:.2f}")
print(f"RMSE: {rmse_avg:.2f}")
print(f"R2: {r2_avg:.2f}")
print(f"Adjusted R2: {adj_r2_avg:.2f}")


***** Overall Averages *****
MAE: 81.92
MSE: 62275.42
RMSE: 170.34
R2: 0.96
Adjusted R2: 0.96


In [88]:
# Get feature importances
importances = rf.feature_importances_

# Create a list of tuples containing the feature and its importance
feature_importances = list(zip(modelfeatures, importances))

# Sort the features by importance
sorted_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)

# Print feature importances
for i, (feature, importance) in enumerate(sorted_importances):
    print(f"{i+1}. {feature} ({importance})")

1. hour (0.8265013081308239)
2. day (0.12153235398846549)
3. month (0.032234816395974905)
4. week_of_month (0.019731521484735746)


Overall performance of random forests:

**Overall Averages**

MAE: 81.92

MSE: 62275.42

RMSE: 170.34

R2: 0.96

Adjusted R2: 0.96

***

## Testing with linear regression

In [89]:
from sklearn.linear_model import LinearRegression

# Create models dictionary
models_dict = {}

for ID in zones_dict:
    # Extract input and output data from zones_dict
    X = zones_dict[ID][modelfeatures].values
    y = zones_dict[ID]['transport_total'].values

    print("******* Zone: %s *******" % ID)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    print("Using %s rows and %s columns." % X_train.shape)

    # Create and fit linear regression model
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    print("Testing on %s rows." % y_test.shape[0])
    # Store model in dictionary
    models_dict[ID] = lr


******* Zone: 4 *******
Using 7119 rows and 4 columns.
Testing on 3052 rows.
******* Zone: 12 *******
Using 5798 rows and 4 columns.
Testing on 2486 rows.
******* Zone: 13 *******
Using 7077 rows and 4 columns.
Testing on 3033 rows.
******* Zone: 24 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 41 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 42 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 43 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 45 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 48 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 50 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 68 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 74 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 75 *******
Using 7123 rows and 4 columns.
Testing o

In [90]:
results_dict = {}

for ID in models_dict:

    # Extract input and output data
    X_test = zones_dict[ID][modelfeatures].values
    y_test = zones_dict[ID]['transport_total']

    # Make predictions on test set
    y_pred = models_dict[ID].predict(X_test)

    # Convert predictions to whole positive numbers
    y_pred = np.round(y_pred).astype(int)
    y_pred = np.abs(y_pred)

    # Calculate absolute error and percentage difference
    abs_error = np.abs(y_test - y_pred)
    pct_error = abs_error / y_test * 100

    # Store results in dictionary
    results_dict[ID] = {'y_test': y_test, 'y_pred': y_pred, 'abs_error': abs_error, 'pct_error': pct_error}

    # Print results for each station
    print(f"***** Zone {ID} *****")
    actual_vs_predicted = pd.concat([pd.DataFrame(y_test.values, columns=['Actual']), pd.DataFrame(y_pred, columns=['Predicted']), pd.DataFrame(pct_error.values, columns=['% Difference'])], axis=1)
    print(actual_vs_predicted)


***** Zone 4 *****
       Actual  Predicted  % Difference
0        31.0         25     19.354839
1        13.0         30    130.769231
2         9.0         35    288.888889
3         3.0         40   1233.333333
4         6.0         45    650.000000
...       ...        ...           ...
10166     5.0         77   1440.000000
10167     2.0         32   1500.000000
10168     4.0         28    600.000000
10169     2.0         41   1950.000000
10170     2.0         35   1650.000000

[10171 rows x 3 columns]
***** Zone 12 *****
      Actual  Predicted  % Difference
0        3.0          7    133.333333
1        5.0          7     40.000000
2        2.0          8    300.000000
3        1.0          9    800.000000
4        4.0          9    125.000000
...      ...        ...           ...
8279     3.0         12    300.000000
8280     1.0          5    400.000000
8281     1.0         12   1100.000000
8282     1.0         13   1200.000000
8283     1.0         13   1200.000000

[8284 rows

In [91]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Initialize dictionaries and variables to store metrics
mae_dict = {}
mse_dict = {}
rmse_dict = {}
r2_dict = {}
adj_r2_dict = {}
mae_total = 0
mse_total = 0
rmse_total = 0
r2_total = 0
adj_r2_total = 0
num_zones = len(results_dict)

# Iterate over station IDs in results_dict
for ID in results_dict:
    
    # Extract test labels and predictions
    y_test = results_dict[ID]['y_test']
    y_pred = results_dict[ID]['y_pred']
    
    # Calculate MAE and store in dictionary
    mae = mean_absolute_error(y_test, y_pred)
    mae_dict[ID] = mae
    mae_total += mae
    
    # Calculate MSE and store in dictionary
    mse = mean_squared_error(y_test, y_pred)
    mse_dict[ID] = mse
    mse_total += mse
    
    # Calculate RMSE and store in dictionary
    rmse = np.sqrt(mse)
    rmse_dict[ID] = rmse
    rmse_total += rmse
    
    # Calculate R2 and store in dictionary
    r2 = r2_score(y_test, y_pred)
    r2_dict[ID] = r2
    r2_total += r2
    
    # Calculate adjusted R2 and store in dictionary
    n = len(y_test)
    p = len(modelfeatures)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    adj_r2_dict[ID] = adj_r2
    adj_r2_total += adj_r2
    
    # # Print results for each station
    # print(f"***** Station {ID} *****")
    # print(f"MAE: {mae:.2f}")
    # print(f"MSE: {mse:.2f}")
    # print(f"RMSE: {rmse:.2f}")
    # print(f"R2: {r2:.2f}")
    # print(f"Adjusted R2: {adj_r2:.2f}")
    
# Calculate overall averages
mae_avg = mae_total / num_zones
mse_avg = mse_total / num_zones
rmse_avg = rmse_total / num_zones
r2_avg = r2_total / num_zones
adj_r2_avg = adj_r2_total / num_zones

# Print overall averages
print("***** Overall Averages *****")
print(f"MAE: {mae_avg:.2f}")
print(f"MSE: {mse_avg:.2f}")
print(f"RMSE: {rmse_avg:.2f}")
print(f"R2: {r2_avg:.2f}")
print(f"Adjusted R2: {adj_r2_avg:.2f}")

***** Overall Averages *****
MAE: 616.40
MSE: 1396393.48
RMSE: 836.68
R2: 0.27
Adjusted R2: 0.27


**Overall linear regression performance**

Overall Averages

MAE: 616.40

MSE: 1396393.48

RMSE: 836.68

R2: 0.27

Adjusted R2: 0.27

***

## Testing Decision Trees


In [92]:
from sklearn.tree import DecisionTreeRegressor

# Create models dictionary
models_dict = {}

for ID in zones_dict:
    # Extract input and output data from zones_dict
    X = zones_dict[ID][modelfeatures].values
    y = zones_dict[ID]['transport_total'].values

    print("******* Zone: %s *******" % ID)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    print("Using %s rows and %s columns." % X_train.shape)

    # Create and fit decision tree regressor model
    dt = DecisionTreeRegressor()
    dt.fit(X_train, y_train)

    print("Testing on %s rows." % y_test.shape[0])
    # Store model in dictionary
    models_dict[ID] = dt


******* Zone: 4 *******
Using 7119 rows and 4 columns.
Testing on 3052 rows.
******* Zone: 12 *******
Using 5798 rows and 4 columns.
Testing on 2486 rows.
******* Zone: 13 *******
Using 7077 rows and 4 columns.
Testing on 3033 rows.
******* Zone: 24 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 41 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 42 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 43 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 45 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 48 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 50 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 68 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 74 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 75 *******
Using 7123 rows and 4 columns.
Testing o

In [93]:
results_dict = {}

for ID in models_dict:

    # Extract input and output data
    X_test = zones_dict[ID][modelfeatures].values
    y_test = zones_dict[ID]['transport_total']

    # Make predictions on test set
    y_pred = models_dict[ID].predict(X_test)

    # Convert predictions to whole positive numbers
    y_pred = np.round(y_pred).astype(int)
    y_pred = np.abs(y_pred)

    # Calculate absolute error and percentage difference
    abs_error = np.abs(y_test - y_pred)
    pct_error = abs_error / y_test * 100

    # Store results in dictionary
    results_dict[ID] = {'y_test': y_test, 'y_pred': y_pred, 'abs_error': abs_error, 'pct_error': pct_error}

    # Print results for each station
    print(f"***** Zone {ID} *****")
    actual_vs_predicted = pd.concat([pd.DataFrame(y_test.values, columns=['Actual']), pd.DataFrame(y_pred, columns=['Predicted']), pd.DataFrame(pct_error.values, columns=['% Difference'])], axis=1)
    print(actual_vs_predicted)


***** Zone 4 *****
       Actual  Predicted  % Difference
0        31.0         36     16.129032
1        13.0         13      0.000000
2         9.0         10     11.111111
3         3.0          4     33.333333
4         6.0          6      0.000000
...       ...        ...           ...
10166     5.0          5      0.000000
10167     2.0          2      0.000000
10168     4.0          4      0.000000
10169     2.0          6    200.000000
10170     2.0         12    500.000000

[10171 rows x 3 columns]
***** Zone 12 *****
      Actual  Predicted  % Difference
0        3.0          3      0.000000
1        5.0         10    100.000000
2        2.0          4    100.000000
3        1.0          2    100.000000
4        4.0          4      0.000000
...      ...        ...           ...
8279     3.0          4     33.333333
8280     1.0          1      0.000000
8281     1.0          1      0.000000
8282     1.0          1      0.000000
8283     1.0          4    300.000000

[8284 rows

In [94]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Initialize dictionaries and variables to store metrics
mae_dict = {}
mse_dict = {}
rmse_dict = {}
r2_dict = {}
adj_r2_dict = {}
mae_total = 0
mse_total = 0
rmse_total = 0
r2_total = 0
adj_r2_total = 0
num_zones = len(results_dict)

# Iterate over station IDs in results_dict
for ID in results_dict:
    
    # Extract test labels and predictions
    y_test = results_dict[ID]['y_test']
    y_pred = results_dict[ID]['y_pred']
    
    # Calculate MAE and store in dictionary
    mae = mean_absolute_error(y_test, y_pred)
    mae_dict[ID] = mae
    mae_total += mae
    
    # Calculate MSE and store in dictionary
    mse = mean_squared_error(y_test, y_pred)
    mse_dict[ID] = mse
    mse_total += mse
    
    # Calculate RMSE and store in dictionary
    rmse = np.sqrt(mse)
    rmse_dict[ID] = rmse
    rmse_total += rmse
    
    # Calculate R2 and store in dictionary
    r2 = r2_score(y_test, y_pred)
    r2_dict[ID] = r2
    r2_total += r2
    
    # Calculate adjusted R2 and store in dictionary
    n = len(y_test)
    p = len(modelfeatures)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    adj_r2_dict[ID] = adj_r2
    adj_r2_total += adj_r2
    
    # # Print results for each station
    # print(f"***** Station {ID} *****")
    # print(f"MAE: {mae:.2f}")
    # print(f"MSE: {mse:.2f}")
    # print(f"RMSE: {rmse:.2f}")
    # print(f"R2: {r2:.2f}")
    # print(f"Adjusted R2: {adj_r2:.2f}")
    
# Calculate overall averages
mae_avg = mae_total / num_zones
mse_avg = mse_total / num_zones
rmse_avg = rmse_total / num_zones
r2_avg = r2_total / num_zones
adj_r2_avg = adj_r2_total / num_zones

# Print overall averages
print("***** Overall Averages *****")
print(f"MAE: {mae_avg:.2f}")
print(f"MSE: {mse_avg:.2f}")
print(f"RMSE: {rmse_avg:.2f}")
print(f"R2: {r2_avg:.2f}")
print(f"Adjusted R2: {adj_r2_avg:.2f}")

***** Overall Averages *****
MAE: 66.76
MSE: 73993.18
RMSE: 186.66
R2: 0.95
Adjusted R2: 0.95


**Decision Trees Performance**

Overall Averages

MAE: 66.76

MSE: 73993.18

RMSE: 186.66

R2: 0.95

Adjusted R2: 0.95

Comparable performance to that of random forests

***

## Testing Gradient Boosting


In [95]:
from sklearn.ensemble import GradientBoostingRegressor

# Create models dictionary
models_dict = {}

for ID in zones_dict:
    # Extract input and output data from zones_dict
    X = zones_dict[ID][modelfeatures].values
    y = zones_dict[ID]['transport_total'].values

    print("******* Zone: %s *******" % ID)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    print("Using %s rows and %s columns." % X_train.shape)

    # Create and fit gradient boosting regressor model
    gb = GradientBoostingRegressor()
    gb.fit(X_train, y_train)

    print("Testing on %s rows." % y_test.shape[0])
    # Store model in dictionary
    models_dict[ID] = gb

# Iterate over LocationIDs in models_dict
results_dict = {}


******* Zone: 4 *******
Using 7119 rows and 4 columns.
Testing on 3052 rows.
******* Zone: 12 *******
Using 5798 rows and 4 columns.
Testing on 2486 rows.
******* Zone: 13 *******
Using 7077 rows and 4 columns.
Testing on 3033 rows.
******* Zone: 24 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 41 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 42 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 43 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 45 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 48 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 50 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 68 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 74 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 75 *******
Using 7123 rows and 4 columns.
Testing o

In [96]:
results_dict = {}

for ID in models_dict:

    # Extract input and output data
    X_test = zones_dict[ID][modelfeatures].values
    y_test = zones_dict[ID]['transport_total']

    # Make predictions on test set
    y_pred = models_dict[ID].predict(X_test)

    # Convert predictions to whole positive numbers
    y_pred = np.round(y_pred).astype(int)
    y_pred = np.abs(y_pred)

    # Calculate absolute error and percentage difference
    abs_error = np.abs(y_test - y_pred)
    pct_error = abs_error / y_test * 100

    # Store results in dictionary
    results_dict[ID] = {'y_test': y_test, 'y_pred': y_pred, 'abs_error': abs_error, 'pct_error': pct_error}

    # Print results for each station
    print(f"***** Zone {ID} *****")
    actual_vs_predicted = pd.concat([pd.DataFrame(y_test.values, columns=['Actual']), pd.DataFrame(y_pred, columns=['Predicted']), pd.DataFrame(pct_error.values, columns=['% Difference'])], axis=1)
    print(actual_vs_predicted)


***** Zone 4 *****
       Actual  Predicted  % Difference
0        31.0         34      9.677419
1        13.0         17     30.769231
2         9.0         10     11.111111
3         3.0          3      0.000000
4         6.0          2     66.666667
...       ...        ...           ...
10166     5.0          6     20.000000
10167     2.0          6    200.000000
10168     4.0         17    325.000000
10169     2.0         10    400.000000
10170     2.0          9    350.000000

[10171 rows x 3 columns]
***** Zone 12 *****
      Actual  Predicted  % Difference
0        3.0          3      0.000000
1        5.0          7     40.000000
2        2.0          8    300.000000
3        1.0          9    800.000000
4        4.0          9    125.000000
...      ...        ...           ...
8279     3.0          2     33.333333
8280     1.0          2    100.000000
8281     1.0          4    300.000000
8282     1.0          3    200.000000
8283     1.0          3    200.000000

[8284 rows

In [97]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Initialize dictionaries and variables to store metrics
mae_dict = {}
mse_dict = {}
rmse_dict = {}
r2_dict = {}
adj_r2_dict = {}
mae_total = 0
mse_total = 0
rmse_total = 0
r2_total = 0
adj_r2_total = 0
num_zones = len(results_dict)

# Iterate over station IDs in results_dict
for ID in results_dict:
    
    # Extract test labels and predictions
    y_test = results_dict[ID]['y_test']
    y_pred = results_dict[ID]['y_pred']
    
    # Calculate MAE and store in dictionary
    mae = mean_absolute_error(y_test, y_pred)
    mae_dict[ID] = mae
    mae_total += mae
    
    # Calculate MSE and store in dictionary
    mse = mean_squared_error(y_test, y_pred)
    mse_dict[ID] = mse
    mse_total += mse
    
    # Calculate RMSE and store in dictionary
    rmse = np.sqrt(mse)
    rmse_dict[ID] = rmse
    rmse_total += rmse
    
    # Calculate R2 and store in dictionary
    r2 = r2_score(y_test, y_pred)
    r2_dict[ID] = r2
    r2_total += r2
    
    # Calculate adjusted R2 and store in dictionary
    n = len(y_test)
    p = len(modelfeatures)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    adj_r2_dict[ID] = adj_r2
    adj_r2_total += adj_r2
    
    # # Print results for each station
    # print(f"***** Station {ID} *****")
    # print(f"MAE: {mae:.2f}")
    # print(f"MSE: {mse:.2f}")
    # print(f"RMSE: {rmse:.2f}")
    # print(f"R2: {r2:.2f}")
    # print(f"Adjusted R2: {adj_r2:.2f}")
    
# Calculate overall averages
mae_avg = mae_total / num_zones
mse_avg = mse_total / num_zones
rmse_avg = rmse_total / num_zones
r2_avg = r2_total / num_zones
adj_r2_avg = adj_r2_total / num_zones

# Print overall averages
print("***** Overall Averages *****")
print(f"MAE: {mae_avg:.2f}")
print(f"MSE: {mse_avg:.2f}")
print(f"RMSE: {rmse_avg:.2f}")
print(f"R2: {r2_avg:.2f}")
print(f"Adjusted R2: {adj_r2_avg:.2f}")

***** Overall Averages *****
MAE: 162.42
MSE: 133733.82
RMSE: 259.10
R2: 0.89
Adjusted R2: 0.89


**Gradient Boosting Performance**

Overall Averages

MAE: 162.42

MSE: 133733.82

RMSE: 259.10

R2: 0.89

Adjusted R2: 0.89

Decent performance, better than linear regression but worse than random forests and decision trees

***

## Testing K-Nearest Neighbours

In [98]:
from sklearn.neighbors import KNeighborsRegressor

# Create models dictionary
models_dict = {}

for ID in zones_dict:
    # Extract input and output data from zones_dict
    X = zones_dict[ID][modelfeatures].values
    y = zones_dict[ID]['transport_total'].values

    print("******* Zone: %s *******" % ID)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    print("Using %s rows and %s columns." % X_train.shape)

    # Create and fit k-nearest neighbors regressor model
    knn = KNeighborsRegressor()
    knn.fit(X_train, y_train)

    print("Testing on %s rows." % y_test.shape[0])
    # Store model in dictionary
    models_dict[ID] = knn

# Iterate over LocationIDs in models_dict
results_dict = {}


******* Zone: 4 *******
Using 7119 rows and 4 columns.
Testing on 3052 rows.
******* Zone: 12 *******
Using 5798 rows and 4 columns.
Testing on 2486 rows.
******* Zone: 13 *******
Using 7077 rows and 4 columns.
Testing on 3033 rows.
******* Zone: 24 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 41 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 42 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 43 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 45 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 48 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 50 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 68 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 74 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 75 *******
Using 7123 rows and 4 columns.
Testing o

In [99]:
results_dict = {}

for ID in models_dict:

    # Extract input and output data
    X_test = zones_dict[ID][modelfeatures].values
    y_test = zones_dict[ID]['transport_total']

    # Make predictions on test set
    y_pred = models_dict[ID].predict(X_test)

    # Convert predictions to whole positive numbers
    y_pred = np.round(y_pred).astype(int)
    y_pred = np.abs(y_pred)

    # Calculate absolute error and percentage difference
    abs_error = np.abs(y_test - y_pred)
    pct_error = abs_error / y_test * 100

    # Store results in dictionary
    results_dict[ID] = {'y_test': y_test, 'y_pred': y_pred, 'abs_error': abs_error, 'pct_error': pct_error}

    # Print results for each station
    print(f"***** Zone {ID} *****")
    actual_vs_predicted = pd.concat([pd.DataFrame(y_test.values, columns=['Actual']), pd.DataFrame(y_pred, columns=['Predicted']), pd.DataFrame(pct_error.values, columns=['% Difference'])], axis=1)
    print(actual_vs_predicted)


***** Zone 4 *****
       Actual  Predicted  % Difference
0        31.0         33      6.451613
1        13.0         18     38.461538
2         9.0         11     22.222222
3         3.0          7    133.333333
4         6.0          6      0.000000
...       ...        ...           ...
10166     5.0         11    120.000000
10167     2.0          8    300.000000
10168     4.0         19    375.000000
10169     2.0          8    300.000000
10170     2.0         13    550.000000

[10171 rows x 3 columns]
***** Zone 12 *****
      Actual  Predicted  % Difference
0        3.0          2     33.333333
1        5.0          7     40.000000
2        2.0          6    200.000000
3        1.0          5    400.000000
4        4.0          6     50.000000
...      ...        ...           ...
8279     3.0          3      0.000000
8280     1.0          2    100.000000
8281     1.0          3    200.000000
8282     1.0          2    100.000000
8283     1.0          4    300.000000

[8284 rows

This took 1 minute 20 seconds to run (tends not to be the longest part of model formation)

In [100]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Initialize dictionaries and variables to store metrics
mae_dict = {}
mse_dict = {}
rmse_dict = {}
r2_dict = {}
adj_r2_dict = {}
mae_total = 0
mse_total = 0
rmse_total = 0
r2_total = 0
adj_r2_total = 0
num_zones = len(results_dict)

# Iterate over station IDs in results_dict
for ID in results_dict:
    
    # Extract test labels and predictions
    y_test = results_dict[ID]['y_test']
    y_pred = results_dict[ID]['y_pred']
    
    # Calculate MAE and store in dictionary
    mae = mean_absolute_error(y_test, y_pred)
    mae_dict[ID] = mae
    mae_total += mae
    
    # Calculate MSE and store in dictionary
    mse = mean_squared_error(y_test, y_pred)
    mse_dict[ID] = mse
    mse_total += mse
    
    # Calculate RMSE and store in dictionary
    rmse = np.sqrt(mse)
    rmse_dict[ID] = rmse
    rmse_total += rmse
    
    # Calculate R2 and store in dictionary
    r2 = r2_score(y_test, y_pred)
    r2_dict[ID] = r2
    r2_total += r2
    
    # Calculate adjusted R2 and store in dictionary
    n = len(y_test)
    p = len(modelfeatures)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    adj_r2_dict[ID] = adj_r2
    adj_r2_total += adj_r2
    
    # # Print results for each station
    # print(f"***** Station {ID} *****")
    # print(f"MAE: {mae:.2f}")
    # print(f"MSE: {mse:.2f}")
    # print(f"RMSE: {rmse:.2f}")
    # print(f"R2: {r2:.2f}")
    # print(f"Adjusted R2: {adj_r2:.2f}")
    
# Calculate overall averages
mae_avg = mae_total / num_zones
mse_avg = mse_total / num_zones
rmse_avg = rmse_total / num_zones
r2_avg = r2_total / num_zones
adj_r2_avg = adj_r2_total / num_zones

# Print overall averages
print("***** Overall Averages *****")
print(f"MAE: {mae_avg:.2f}")
print(f"MSE: {mse_avg:.2f}")
print(f"RMSE: {rmse_avg:.2f}")
print(f"R2: {r2_avg:.2f}")
print(f"Adjusted R2: {adj_r2_avg:.2f}")

***** Overall Averages *****
MAE: 128.35
MSE: 97165.08
RMSE: 218.11
R2: 0.93
Adjusted R2: 0.93


**K-Nearest Neighbours Performance**

Overall Averages

MAE: 128.35

MSE: 97165.08

RMSE: 218.11

R2: 0.93

Adjusted R2: 0.93

***

## Testing XGBoost

In [101]:
from xgboost import XGBRegressor

# Create models dictionary
models_dict = {}

for ID in zones_dict:
    # Extract input and output data from zones_dict
    X = zones_dict[ID][modelfeatures].values
    y = zones_dict[ID]['transport_total'].values

    print("******* Zone: %s *******" % ID)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    print("Using %s rows and %s columns." % X_train.shape)

    # Create and fit XGBoost regressor model
    xgb = XGBRegressor()
    xgb.fit(X_train, y_train)

    print("Testing on %s rows." % y_test.shape[0])
    # Store model in dictionary
    models_dict[ID] = xgb

# Iterate over LocationIDs in models_dict
results_dict = {}


******* Zone: 4 *******
Using 7119 rows and 4 columns.
Testing on 3052 rows.
******* Zone: 12 *******
Using 5798 rows and 4 columns.
Testing on 2486 rows.
******* Zone: 13 *******
Using 7077 rows and 4 columns.
Testing on 3033 rows.
******* Zone: 24 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 41 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 42 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 43 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 45 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 48 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 50 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 68 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 74 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 75 *******
Using 7123 rows and 4 columns.
Testing o

Took about 1 minute 20 to run

In [102]:
from xgboost import XGBRegressor

# Create models dictionary
models_dict = {}

for ID in zones_dict:
    # Extract input and output data from zones_dict
    X = zones_dict[ID][modelfeatures].values
    y = zones_dict[ID]['transport_total'].values

    print("******* Zone: %s *******" % ID)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    print("Using %s rows and %s columns." % X_train.shape)

    # Create and fit XGBoost regressor model
    xgb = XGBRegressor()
    xgb.fit(X_train, y_train)

    print("Testing on %s rows." % y_test.shape[0])
    # Store model in dictionary
    models_dict[ID] = xgb

# Iterate over LocationIDs in models_dict
results_dict = {}


******* Zone: 4 *******
Using 7119 rows and 4 columns.
Testing on 3052 rows.
******* Zone: 12 *******
Using 5798 rows and 4 columns.
Testing on 2486 rows.
******* Zone: 13 *******
Using 7077 rows and 4 columns.
Testing on 3033 rows.
******* Zone: 24 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 41 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 42 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 43 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 45 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 48 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 50 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 68 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 74 *******
Using 7123 rows and 4 columns.
Testing on 3053 rows.
******* Zone: 75 *******
Using 7123 rows and 4 columns.
Testing o

In [103]:
results_dict = {}

for ID in models_dict:

    # Extract input and output data
    X_test = zones_dict[ID][modelfeatures].values
    y_test = zones_dict[ID]['transport_total']

    # Make predictions on test set
    y_pred = models_dict[ID].predict(X_test)

    # Convert predictions to whole positive numbers
    y_pred = np.round(y_pred).astype(int)
    y_pred = np.abs(y_pred)

    # Calculate absolute error and percentage difference
    abs_error = np.abs(y_test - y_pred)
    pct_error = abs_error / y_test * 100

    # Store results in dictionary
    results_dict[ID] = {'y_test': y_test, 'y_pred': y_pred, 'abs_error': abs_error, 'pct_error': pct_error}

    # Print results for each station
    print(f"***** Zone {ID} *****")
    actual_vs_predicted = pd.concat([pd.DataFrame(y_test.values, columns=['Actual']), pd.DataFrame(y_pred, columns=['Predicted']), pd.DataFrame(pct_error.values, columns=['% Difference'])], axis=1)
    print(actual_vs_predicted)


***** Zone 4 *****
       Actual  Predicted  % Difference
0        31.0         32      3.225806
1        13.0         16     23.076923
2         9.0          8     11.111111
3         3.0          4     33.333333
4         6.0          3     50.000000
...       ...        ...           ...
10166     5.0          4     20.000000
10167     2.0          8    300.000000
10168     4.0          8    100.000000
10169     2.0          2      0.000000
10170     2.0         14    600.000000

[10171 rows x 3 columns]
***** Zone 12 *****
      Actual  Predicted  % Difference
0        3.0          3           0.0
1        5.0          9          80.0
2        2.0          6         200.0
3        1.0          3         200.0
4        4.0          5          25.0
...      ...        ...           ...
8279     3.0          3           0.0
8280     1.0          2         100.0
8281     1.0          3         200.0
8282     1.0          2         100.0
8283     1.0          2         100.0

[8284 rows

In [104]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Initialize dictionaries and variables to store metrics
mae_dict = {}
mse_dict = {}
rmse_dict = {}
r2_dict = {}
adj_r2_dict = {}
mae_total = 0
mse_total = 0
rmse_total = 0
r2_total = 0
adj_r2_total = 0
num_zones = len(results_dict)

# Iterate over station IDs in results_dict
for ID in results_dict:
    
    # Extract test labels and predictions
    y_test = results_dict[ID]['y_test']
    y_pred = results_dict[ID]['y_pred']
    
    # Calculate MAE and store in dictionary
    mae = mean_absolute_error(y_test, y_pred)
    mae_dict[ID] = mae
    mae_total += mae
    
    # Calculate MSE and store in dictionary
    mse = mean_squared_error(y_test, y_pred)
    mse_dict[ID] = mse
    mse_total += mse
    
    # Calculate RMSE and store in dictionary
    rmse = np.sqrt(mse)
    rmse_dict[ID] = rmse
    rmse_total += rmse
    
    # Calculate R2 and store in dictionary
    r2 = r2_score(y_test, y_pred)
    r2_dict[ID] = r2
    r2_total += r2
    
    # Calculate adjusted R2 and store in dictionary
    n = len(y_test)
    p = len(modelfeatures)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    adj_r2_dict[ID] = adj_r2
    adj_r2_total += adj_r2
    
    # # Print results for each station
    # print(f"***** Station {ID} *****")
    # print(f"MAE: {mae:.2f}")
    # print(f"MSE: {mse:.2f}")
    # print(f"RMSE: {rmse:.2f}")
    # print(f"R2: {r2:.2f}")
    # print(f"Adjusted R2: {adj_r2:.2f}")
    
# Calculate overall averages
mae_avg = mae_total / num_zones
mse_avg = mse_total / num_zones
rmse_avg = rmse_total / num_zones
r2_avg = r2_total / num_zones
adj_r2_avg = adj_r2_total / num_zones

# Print overall averages
print("***** Overall Averages *****")
print(f"MAE: {mae_avg:.2f}")
print(f"MSE: {mse_avg:.2f}")
print(f"RMSE: {rmse_avg:.2f}")
print(f"R2: {r2_avg:.2f}")
print(f"Adjusted R2: {adj_r2_avg:.2f}")

***** Overall Averages *****
MAE: 85.27
MSE: 50150.48
RMSE: 154.86
R2: 0.96
Adjusted R2: 0.96


**XGBoost Performance**

Overall Averages

MAE: 85.27

MSE: 50150.48

RMSE: 154.86

R2: 0.96

Adjusted R2: 0.96

Very high performance comparable to that of Random Forests

***

## Model evaluations

|   |  Random Forests | Linear Regression | Decision Trees | Gradient Boosting | K-Nearest Neighbours | XGBoost |
|----------|----------|----------|----------|----------|----------|----------|
|  MAE  |   81.92  |   616.40  |   66.76  |   162.42  |   128.35  |   85.27  |
|  MSE  |   62275.42  |   1396393.48  |   73993.18  |   133733.82  |   97165.08  |   50150.48  |
|  RMSE  |   170.34  |   836.68  |   186.66  |   259.10  |   218.11  |  154.86  |
|  R2  |   0.96  |   0.27  |   0.95  |   0.89  |  0.93  |   0.96  |
|  Adjusted R2  |   0.96  |   0.27  |   0.95  |   0.89  |   0.93  |   0.96  |

As can be seen from the table, the top 3 best performing models are Random Forests, Decision Trees, and XGboost

We will now take these three and examine whether performance can be improved by adding in US Public Holidays as a dummy variables


In [105]:
initial_df = pd.read_parquet('unified_complete.parquet')

# Extract hour, day, and month as separate columns
initial_df['hour'] = initial_df['datetime'].dt.hour
initial_df['day'] = initial_df['datetime'].dt.dayofweek
initial_df['month'] = initial_df['datetime'].dt.month
initial_df['week_of_month'] = (initial_df['datetime'].dt.day - 1) // 7

# Generate dummy variables for hour, day, and month
# df_dummies = pd.get_dummies(initial_df, columns=['hour', 'day', 'month'])
df_dummies = initial_df
df_dummies

,LocationID,datetime,bike_count,passenger_count,ridership,transport_total,hour,day,month,week_of_month
725,4,2022-02-01 00:00:00,11.0,20.0,NaN,31.0,0,1,2,0
726,4,2022-02-01 01:00:00,5.0,8.0,NaN,13.0,1,1,2,0
727,4,2022-02-01 02:00:00,1.0,8.0,NaN,9.0,2,1,2,0
728,4,2022-02-01 03:00:00,2.0,1.0,NaN,3.0,3,1,2,0
729,4,2022-02-01 04:00:00,1.0,5.0,NaN,6.0,4,1,2,0
...,...,...,...,...,...,...,...,...,...,...
800273,261,2023-02-18 06:00:00,NaN,NaN,205.0,205.0,6,5,2,2
800274,261,2023-02-21 04:00:00,NaN,NaN,43.0,43.0,4,1,2,2
800275,261,2023-02-27 03:00:00,NaN,NaN,19.0,19.0,3,0,2,3
800276,261,2023-02-28 04:00:00,NaN,NaN,45.0,45.0,4,1,2,3


In [106]:
from datetime import datetime
import holidays

# Create an instance of the US holidays
us_holidays = holidays.US()

# Iterate over the rows of the df_dummies DataFrame and check if the date is a US public holiday
df_dummies['is_public_holiday'] = df_dummies['datetime'].apply(lambda x: x.date() in us_holidays)

In [107]:
df_dummies

,LocationID,datetime,bike_count,passenger_count,ridership,transport_total,hour,day,month,week_of_month,is_public_holiday
725,4,2022-02-01 00:00:00,11.0,20.0,NaN,31.0,0,1,2,0,False
726,4,2022-02-01 01:00:00,5.0,8.0,NaN,13.0,1,1,2,0,False
727,4,2022-02-01 02:00:00,1.0,8.0,NaN,9.0,2,1,2,0,False
728,4,2022-02-01 03:00:00,2.0,1.0,NaN,3.0,3,1,2,0,False
729,4,2022-02-01 04:00:00,1.0,5.0,NaN,6.0,4,1,2,0,False
...,...,...,...,...,...,...,...,...,...,...,...
800273,261,2023-02-18 06:00:00,NaN,NaN,205.0,205.0,6,5,2,2,False
800274,261,2023-02-21 04:00:00,NaN,NaN,43.0,43.0,4,1,2,2,False
800275,261,2023-02-27 03:00:00,NaN,NaN,19.0,19.0,3,0,2,3,False
800276,261,2023-02-28 04:00:00,NaN,NaN,45.0,45.0,4,1,2,3,False


In [108]:
df_ml = df_dummies
df_dummies.drop(['bike_count', 'passenger_count', 'ridership', 'datetime'], axis=1, inplace=True)
df_ml

,LocationID,transport_total,hour,day,month,week_of_month,is_public_holiday
725,4,31.0,0,1,2,0,False
726,4,13.0,1,1,2,0,False
727,4,9.0,2,1,2,0,False
728,4,3.0,3,1,2,0,False
729,4,6.0,4,1,2,0,False
...,...,...,...,...,...,...,...
800273,261,205.0,6,5,2,2,False
800274,261,43.0,4,1,2,2,False
800275,261,19.0,3,0,2,3,False
800276,261,45.0,4,1,2,3,False


### Random Forests with Holidays

In [109]:
zones_dict = {}


# Iterate over unique LocationIDs in df_ml
for ID in df_ml['LocationID'].unique():
    
    # Select rows with matching LocationID and drop the LocationID column
    df_location = df_ml.loc[df_ml['LocationID'] == ID].drop(['LocationID'], axis=1)
    
    # Reset the index and drop the old index column
    df_location = df_location.reset_index(drop=True)
    
    # Add the dataframe to the zones_dict
    zones_dict[ID] = df_location

print("Zones Dict")
print(zones_dict)

modelfeatures = ['hour', 'day', 'month','week_of_month','is_public_holiday']

# Create models dictionary
models_dict = {}

for ID in zones_dict:
    # Extract input and output data from zones_dict
    X = zones_dict[ID][modelfeatures].values
    y = zones_dict[ID]['transport_total'].values

    print("******* Zone: %s *******" % ID)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    print("Using %s rows and %s columns." % X_train.shape)

    # Create and fit random forest regressor model
    rf = RandomForestRegressor(n_estimators=100)
    rf.fit(X_train, y_train)

    print("Testing on %s rows." % y_test.shape[0])
    # Store model in dictionary
    models_dict[ID] = rf


Zones Dict
{4:        transport_total  hour  day  month  week_of_month  is_public_holiday
0                 31.0     0    1      2              0              False
1                 13.0     1    1      2              0              False
2                  9.0     2    1      2              0              False
3                  3.0     3    1      2              0              False
4                  6.0     4    1      2              0              False
...                ...   ...  ...    ...            ...                ...
10166              5.0     5    6     12              3               True
10167              2.0     2    0      1              0               True
10168              4.0     1    0      1              3              False
10169              2.0     3    1      2              3              False
10170              2.0     2    0      3              0              False

[10171 rows x 6 columns], 12:       transport_total  hour  day  month  week_of_month

In [110]:
results_dict = {}

for ID in models_dict:

    # Extract input and output data
    X_test = zones_dict[ID][modelfeatures].values
    y_test = zones_dict[ID]['transport_total']

    # Make predictions on test set
    y_pred = models_dict[ID].predict(X_test)

    # Convert predictions to whole positive numbers
    y_pred = np.round(y_pred).astype(int)
    y_pred = np.abs(y_pred)

    # Calculate absolute error and percentage difference
    abs_error = np.abs(y_test - y_pred)
    pct_error = abs_error / y_test * 100

    # Store results in dictionary
    results_dict[ID] = {'y_test': y_test, 'y_pred': y_pred, 'abs_error': abs_error, 'pct_error': pct_error}

    # Print results for each station
    print(f"***** Zone {ID} *****")
    actual_vs_predicted = pd.concat([pd.DataFrame(y_test.values, columns=['Actual']), pd.DataFrame(y_pred, columns=['Predicted']), pd.DataFrame(pct_error.values, columns=['% Difference'])], axis=1)
    print(actual_vs_predicted)


***** Zone 4 *****
       Actual  Predicted  % Difference
0        31.0         34      9.677419
1        13.0         14      7.692308
2         9.0         10     11.111111
3         3.0          4     33.333333
4         6.0          6      0.000000
...       ...        ...           ...
10166     5.0          9     80.000000
10167     2.0          8    300.000000
10168     4.0          9    125.000000
10169     2.0          7    250.000000
10170     2.0         12    500.000000

[10171 rows x 3 columns]
***** Zone 12 *****
      Actual  Predicted  % Difference
0        3.0          3      0.000000
1        5.0          9     80.000000
2        2.0          5    150.000000
3        1.0          2    100.000000
4        4.0          4      0.000000
...      ...        ...           ...
8279     3.0          4     33.333333
8280     1.0          1      0.000000
8281     1.0          2    100.000000
8282     1.0          2    100.000000
8283     1.0          4    300.000000

[8284 rows

In [111]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Initialize dictionaries and variables to store metrics
mae_dict = {}
mse_dict = {}
rmse_dict = {}
r2_dict = {}
adj_r2_dict = {}
mae_total = 0
mse_total = 0
rmse_total = 0
r2_total = 0
adj_r2_total = 0
num_zones = len(results_dict)

# Iterate over station IDs in results_dict
for ID in results_dict:
    
    # Extract test labels and predictions
    y_test = results_dict[ID]['y_test']
    y_pred = results_dict[ID]['y_pred']
    
    # Calculate MAE and store in dictionary
    mae = mean_absolute_error(y_test, y_pred)
    mae_dict[ID] = mae
    mae_total += mae
    
    # Calculate MSE and store in dictionary
    mse = mean_squared_error(y_test, y_pred)
    mse_dict[ID] = mse
    mse_total += mse
    
    # Calculate RMSE and store in dictionary
    rmse = np.sqrt(mse)
    rmse_dict[ID] = rmse
    rmse_total += rmse
    
    # Calculate R2 and store in dictionary
    r2 = r2_score(y_test, y_pred)
    r2_dict[ID] = r2
    r2_total += r2
    
    # Calculate adjusted R2 and store in dictionary
    n = len(y_test)
    p = len(modelfeatures)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    adj_r2_dict[ID] = adj_r2
    adj_r2_total += adj_r2
    
    # # Print results for each station
    # print(f"***** Station {ID} *****")
    # print(f"MAE: {mae:.2f}")
    # print(f"MSE: {mse:.2f}")
    # print(f"RMSE: {rmse:.2f}")
    # print(f"R2: {r2:.2f}")
    # print(f"Adjusted R2: {adj_r2:.2f}")
    
# Calculate overall averages
mae_avg = mae_total / num_zones
mse_avg = mse_total / num_zones
rmse_avg = rmse_total / num_zones
r2_avg = r2_total / num_zones
adj_r2_avg = adj_r2_total / num_zones

# Print overall averages
print("***** Overall Averages *****")
print(f"MAE: {mae_avg:.2f}")
print(f"MSE: {mse_avg:.2f}")
print(f"RMSE: {rmse_avg:.2f}")
print(f"R2: {r2_avg:.2f}")
print(f"Adjusted R2: {adj_r2_avg:.2f}")

***** Overall Averages *****
MAE: 76.04
MSE: 50710.99
RMSE: 154.05
R2: 0.96
Adjusted R2: 0.96


### Testing Decision Trees with Public Holidays

In [112]:
from sklearn.tree import DecisionTreeRegressor

# Create models dictionary
models_dict = {}

for ID in zones_dict:
    # Extract input and output data from zones_dict
    X = zones_dict[ID][modelfeatures].values
    y = zones_dict[ID]['transport_total'].values

    print("******* Zone: %s *******" % ID)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    print("Using %s rows and %s columns." % X_train.shape)

    # Create and fit decision tree regressor model
    dt = DecisionTreeRegressor()
    dt.fit(X_train, y_train)

    print("Testing on %s rows." % y_test.shape[0])
    # Store model in dictionary
    models_dict[ID] = dt


******* Zone: 4 *******
Using 7119 rows and 5 columns.
Testing on 3052 rows.
******* Zone: 12 *******
Using 5798 rows and 5 columns.
Testing on 2486 rows.
******* Zone: 13 *******
Using 7077 rows and 5 columns.
Testing on 3033 rows.
******* Zone: 24 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 41 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 42 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 43 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 45 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 48 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 50 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 68 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 74 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 75 *******
Using 7123 rows and 5 columns.
Testing o

In [113]:
results_dict = {}

for ID in models_dict:

    # Extract input and output data
    X_test = zones_dict[ID][modelfeatures].values
    y_test = zones_dict[ID]['transport_total']

    # Make predictions on test set
    y_pred = models_dict[ID].predict(X_test)

    # Convert predictions to whole positive numbers
    y_pred = np.round(y_pred).astype(int)
    y_pred = np.abs(y_pred)

    # Calculate absolute error and percentage difference
    abs_error = np.abs(y_test - y_pred)
    pct_error = abs_error / y_test * 100

    # Store results in dictionary
    results_dict[ID] = {'y_test': y_test, 'y_pred': y_pred, 'abs_error': abs_error, 'pct_error': pct_error}

    # Print results for each station
    print(f"***** Zone {ID} *****")
    actual_vs_predicted = pd.concat([pd.DataFrame(y_test.values, columns=['Actual']), pd.DataFrame(y_pred, columns=['Predicted']), pd.DataFrame(pct_error.values, columns=['% Difference'])], axis=1)
    print(actual_vs_predicted)


***** Zone 4 *****
       Actual  Predicted  % Difference
0        31.0         36     16.129032
1        13.0         13      0.000000
2         9.0         10     11.111111
3         3.0          4     33.333333
4         6.0          6      0.000000
...       ...        ...           ...
10166     5.0          5      0.000000
10167     2.0          2      0.000000
10168     4.0          4      0.000000
10169     2.0          6    200.000000
10170     2.0         12    500.000000

[10171 rows x 3 columns]
***** Zone 12 *****
      Actual  Predicted  % Difference
0        3.0          3      0.000000
1        5.0         10    100.000000
2        2.0          4    100.000000
3        1.0          2    100.000000
4        4.0          4      0.000000
...      ...        ...           ...
8279     3.0          4     33.333333
8280     1.0          1      0.000000
8281     1.0          1      0.000000
8282     1.0          1      0.000000
8283     1.0          4    300.000000

[8284 rows

In [114]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Initialize dictionaries and variables to store metrics
mae_dict = {}
mse_dict = {}
rmse_dict = {}
r2_dict = {}
adj_r2_dict = {}
mae_total = 0
mse_total = 0
rmse_total = 0
r2_total = 0
adj_r2_total = 0
num_zones = len(results_dict)

# Iterate over station IDs in results_dict
for ID in results_dict:
    
    # Extract test labels and predictions
    y_test = results_dict[ID]['y_test']
    y_pred = results_dict[ID]['y_pred']
    
    # Calculate MAE and store in dictionary
    mae = mean_absolute_error(y_test, y_pred)
    mae_dict[ID] = mae
    mae_total += mae
    
    # Calculate MSE and store in dictionary
    mse = mean_squared_error(y_test, y_pred)
    mse_dict[ID] = mse
    mse_total += mse
    
    # Calculate RMSE and store in dictionary
    rmse = np.sqrt(mse)
    rmse_dict[ID] = rmse
    rmse_total += rmse
    
    # Calculate R2 and store in dictionary
    r2 = r2_score(y_test, y_pred)
    r2_dict[ID] = r2
    r2_total += r2
    
    # Calculate adjusted R2 and store in dictionary
    n = len(y_test)
    p = len(modelfeatures)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    adj_r2_dict[ID] = adj_r2
    adj_r2_total += adj_r2
    
    # # Print results for each station
    # print(f"***** Station {ID} *****")
    # print(f"MAE: {mae:.2f}")
    # print(f"MSE: {mse:.2f}")
    # print(f"RMSE: {rmse:.2f}")
    # print(f"R2: {r2:.2f}")
    # print(f"Adjusted R2: {adj_r2:.2f}")
    
# Calculate overall averages
mae_avg = mae_total / num_zones
mse_avg = mse_total / num_zones
rmse_avg = rmse_total / num_zones
r2_avg = r2_total / num_zones
adj_r2_avg = adj_r2_total / num_zones

# Print overall averages
print("***** Overall Averages *****")
print(f"MAE: {mae_avg:.2f}")
print(f"MSE: {mse_avg:.2f}")
print(f"RMSE: {rmse_avg:.2f}")
print(f"R2: {r2_avg:.2f}")
print(f"Adjusted R2: {adj_r2_avg:.2f}")

***** Overall Averages *****
MAE: 63.12
MSE: 61488.68
RMSE: 169.90
R2: 0.95
Adjusted R2: 0.95


### Testing XGBoost with US Public Holidays

In [115]:
from xgboost import XGBRegressor

# Create models dictionary
models_dict = {}

for ID in zones_dict:
    # Extract input and output data from zones_dict
    X = zones_dict[ID][modelfeatures].values
    y = zones_dict[ID]['transport_total'].values

    print("******* Zone: %s *******" % ID)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    print("Using %s rows and %s columns." % X_train.shape)

    # Create and fit XGBoost regressor model
    xgb = XGBRegressor()
    xgb.fit(X_train, y_train)

    print("Testing on %s rows." % y_test.shape[0])
    # Store model in dictionary
    models_dict[ID] = xgb

# Iterate over LocationIDs in models_dict
results_dict = {}


******* Zone: 4 *******
Using 7119 rows and 5 columns.
Testing on 3052 rows.
******* Zone: 12 *******
Using 5798 rows and 5 columns.
Testing on 2486 rows.
******* Zone: 13 *******
Using 7077 rows and 5 columns.
Testing on 3033 rows.
******* Zone: 24 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 41 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 42 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 43 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 45 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 48 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 50 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 68 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 74 *******
Using 7123 rows and 5 columns.
Testing on 3053 rows.
******* Zone: 75 *******
Using 7123 rows and 5 columns.
Testing o

In [116]:
results_dict = {}

for ID in models_dict:

    # Extract input and output data
    X_test = zones_dict[ID][modelfeatures].values
    y_test = zones_dict[ID]['transport_total']

    # Make predictions on test set
    y_pred = models_dict[ID].predict(X_test)

    # Convert predictions to whole positive numbers
    y_pred = np.round(y_pred).astype(int)
    y_pred = np.abs(y_pred)

    # Calculate absolute error and percentage difference
    abs_error = np.abs(y_test - y_pred)
    pct_error = abs_error / y_test * 100

    # Store results in dictionary
    results_dict[ID] = {'y_test': y_test, 'y_pred': y_pred, 'abs_error': abs_error, 'pct_error': pct_error}

    # Print results for each station
    print(f"***** Zone {ID} *****")
    actual_vs_predicted = pd.concat([pd.DataFrame(y_test.values, columns=['Actual']), pd.DataFrame(y_pred, columns=['Predicted']), pd.DataFrame(pct_error.values, columns=['% Difference'])], axis=1)
    print(actual_vs_predicted)


***** Zone 4 *****
       Actual  Predicted  % Difference
0        31.0         30      3.225806
1        13.0         17     30.769231
2         9.0          8     11.111111
3         3.0          5     66.666667
4         6.0          2     66.666667
...       ...        ...           ...
10166     5.0          7     40.000000
10167     2.0          9    350.000000
10168     4.0          8    100.000000
10169     2.0          2      0.000000
10170     2.0         13    550.000000

[10171 rows x 3 columns]
***** Zone 12 *****
      Actual  Predicted  % Difference
0        3.0          4     33.333333
1        5.0          8     60.000000
2        2.0          6    200.000000
3        1.0          4    300.000000
4        4.0          4      0.000000
...      ...        ...           ...
8279     3.0          4     33.333333
8280     1.0          2    100.000000
8281     1.0          2    100.000000
8282     1.0          2    100.000000
8283     1.0          3    200.000000

[8284 rows

In [117]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Initialize dictionaries and variables to store metrics
mae_dict = {}
mse_dict = {}
rmse_dict = {}
r2_dict = {}
adj_r2_dict = {}
mae_total = 0
mse_total = 0
rmse_total = 0
r2_total = 0
adj_r2_total = 0
num_zones = len(results_dict)

# Iterate over station IDs in results_dict
for ID in results_dict:
    
    # Extract test labels and predictions
    y_test = results_dict[ID]['y_test']
    y_pred = results_dict[ID]['y_pred']
    
    # Calculate MAE and store in dictionary
    mae = mean_absolute_error(y_test, y_pred)
    mae_dict[ID] = mae
    mae_total += mae
    
    # Calculate MSE and store in dictionary
    mse = mean_squared_error(y_test, y_pred)
    mse_dict[ID] = mse
    mse_total += mse
    
    # Calculate RMSE and store in dictionary
    rmse = np.sqrt(mse)
    rmse_dict[ID] = rmse
    rmse_total += rmse
    
    # Calculate R2 and store in dictionary
    r2 = r2_score(y_test, y_pred)
    r2_dict[ID] = r2
    r2_total += r2
    
    # Calculate adjusted R2 and store in dictionary
    n = len(y_test)
    p = len(modelfeatures)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    adj_r2_dict[ID] = adj_r2
    adj_r2_total += adj_r2
    
    # # Print results for each station
    # print(f"***** Station {ID} *****")
    # print(f"MAE: {mae:.2f}")
    # print(f"MSE: {mse:.2f}")
    # print(f"RMSE: {rmse:.2f}")
    # print(f"R2: {r2:.2f}")
    # print(f"Adjusted R2: {adj_r2:.2f}")
    
# Calculate overall averages
mae_avg = mae_total / num_zones
mse_avg = mse_total / num_zones
rmse_avg = rmse_total / num_zones
r2_avg = r2_total / num_zones
adj_r2_avg = adj_r2_total / num_zones

# Print overall averages
print("***** Overall Averages *****")
print(f"MAE: {mae_avg:.2f}")
print(f"MSE: {mse_avg:.2f}")
print(f"RMSE: {rmse_avg:.2f}")
print(f"R2: {r2_avg:.2f}")
print(f"Adjusted R2: {adj_r2_avg:.2f}")

***** Overall Averages *****
MAE: 80.10
MSE: 44020.99
RMSE: 144.89
R2: 0.96
Adjusted R2: 0.96


## Overall evaluation with addition of US Public Holidays

|   |  Random Forests | Random Forests & Holidays | Decision Trees | Decision Trees & Holidays | XGBoost & Holidays | XGBoost |
|----------|----------|----------|----------|----------|----------|----------|
|  MAE  |   81.92  |   76.04  |   66.76  |   63.12  |   80.10  |   85.27  |
|  MSE  |   62275.42  |   50710.99  |   73993.18  |   61488.68 |   44020.99  |   50150.48  |
|  RMSE  |   170.34  |   154.05  |   186.66  |   169.90  |   144.89  |  154.86  |
|  R2  |   0.96  |   0.96  |   0.95  |   0.95  |  0.96  |   0.96  |
|  Adjusted R2  |   0.96  |   0.96  |   0.95  |   0.95  |   0.96  |   0.96  |

All three models experienced similar **incremental** improvements and share similar performance. Their MAEs, MSEs, & RMSEs improved. 

However for ease of use on the website we wish to generate predictions for the entire year. In order to make this process as seamless as possible we will exclude Public Holidays from this (as the performance gains does not justify the modest improvements to the models performance)

We will pickle the Random Forests models

***


In [118]:
initial_df = pd.read_parquet('unified_complete.parquet')

# Extract hour, day, and month as separate columns
initial_df['hour'] = initial_df['datetime'].dt.hour
initial_df['day'] = initial_df['datetime'].dt.dayofweek
initial_df['month'] = initial_df['datetime'].dt.month
initial_df['week_of_month'] = (initial_df['datetime'].dt.day - 1) // 7

df_dummies = initial_df

df_dummies.drop(['bike_count', 'passenger_count', 'ridership', 'datetime'], axis=1, inplace=True)

df_ml = df_dummies

df_ml

,LocationID,transport_total,hour,day,month,week_of_month
725,4,31.0,0,1,2,0
726,4,13.0,1,1,2,0
727,4,9.0,2,1,2,0
728,4,3.0,3,1,2,0
729,4,6.0,4,1,2,0
...,...,...,...,...,...,...
800273,261,205.0,6,5,2,2
800274,261,43.0,4,1,2,2
800275,261,19.0,3,0,2,3
800276,261,45.0,4,1,2,3


In [121]:
from sklearn.ensemble import RandomForestRegressor
import pickle

zones_dict = {}

# Iterate over unique station IDs in merged_df
for ID in df_ml['LocationID'].unique():
    
    # Select rows with matching LocationID and drop the LocationID column
    df_location = df_ml.loc[df_ml['LocationID'] == ID].drop(['LocationID'], axis=1)
    
    # Reset the index and drop the old index column
    df_location = df_location.reset_index(drop=True)
    
    # Add the dataframe to the zones_dict
    zones_dict[ID] = df_location

print("Zones Dict")
print(zones_dict)

modelfeatures = ['hour', 'day', 'month', 'week_of_month']

# Create models dictionary
models_dict = {}

for ID in zones_dict:
    # Extract input and output data from stations_dict
    X = zones_dict[ID][modelfeatures].values
    y = zones_dict[ID]['transport_total'].values

    print("******* Station: %s *******" % ID)

    # Split data into training and testing sets
    X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.3, random_state=0)

    print("Using %s rows and %s columns." % X_train.shape)

    # Create and fit random forest regressor model
    rf = RandomForestRegressor(n_estimators=100)
    rf.fit(X_train, y_train)

    print("Testing on %s rows." % y_test.shape[0])
    
    print("Pickling…")
    filename = f"pickle_models/model_{ID}.pkl"
    with open(filename, 'wb') as handle:
        pickle.dump(rf, handle)

    # Store model in dictionary
    models_dict[ID] = rf


Zones Dict
{4:        transport_total  hour  day  month  week_of_month
0                 31.0     0    1      2              0
1                 13.0     1    1      2              0
2                  9.0     2    1      2              0
3                  3.0     3    1      2              0
4                  6.0     4    1      2              0
...                ...   ...  ...    ...            ...
10166              5.0     5    6     12              3
10167              2.0     2    0      1              0
10168              4.0     1    0      1              3
10169              2.0     3    1      2              3
10170              2.0     2    0      3              0

[10171 rows x 5 columns], 12:       transport_total  hour  day  month  week_of_month
0                 3.0     7    1      2              0
1                 5.0     8    1      2              0
2                 2.0    10    1      2              0
3                 1.0    13    1      2              0
4      

In [122]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Initialize dictionaries and variables to store metrics
mae_dict = {}
mse_dict = {}
rmse_dict = {}
r2_dict = {}
adj_r2_dict = {}
mae_total = 0
mse_total = 0
rmse_total = 0
r2_total = 0
adj_r2_total = 0
num_zones = len(results_dict)

# Iterate over station IDs in results_dict
for ID in results_dict:
    
    # Extract test labels and predictions
    y_test = results_dict[ID]['y_test']
    y_pred = results_dict[ID]['y_pred']
    
    # Calculate MAE and store in dictionary
    mae = mean_absolute_error(y_test, y_pred)
    mae_dict[ID] = mae
    mae_total += mae
    
    # Calculate MSE and store in dictionary
    mse = mean_squared_error(y_test, y_pred)
    mse_dict[ID] = mse
    mse_total += mse
    
    # Calculate RMSE and store in dictionary
    rmse = np.sqrt(mse)
    rmse_dict[ID] = rmse
    rmse_total += rmse
    
    # Calculate R2 and store in dictionary
    r2 = r2_score(y_test, y_pred)
    r2_dict[ID] = r2
    r2_total += r2
    
    # Calculate adjusted R2 and store in dictionary
    n = len(y_test)
    p = len(modelfeatures)
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
    adj_r2_dict[ID] = adj_r2
    adj_r2_total += adj_r2
    
    # Print results for each station
    print(f"***** Station {ID} *****")
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R2: {r2:.2f}")
    print(f"Adjusted R2: {adj_r2:.2f}")
    
# Calculate overall averages
mae_avg = mae_total / num_zones
mse_avg = mse_total / num_zones
rmse_avg = rmse_total / num_zones
r2_avg = r2_total / num_zones
adj_r2_avg = adj_r2_total / num_zones

# Print overall averages
print("***** Overall Averages *****")
print(f"MAE: {mae_avg:.2f}")
print(f"MSE: {mse_avg:.2f}")
print(f"RMSE: {rmse_avg:.2f}")
print(f"R2: {r2_avg:.2f}")
print(f"Adjusted R2: {adj_r2_avg:.2f}")

***** Station 4 *****
MAE: 10.50
MSE: 218.20
RMSE: 14.77
R2: 0.93
Adjusted R2: 0.93
***** Station 12 *****
MAE: 3.19
MSE: 23.66
RMSE: 4.86
R2: 0.81
Adjusted R2: 0.81
***** Station 13 *****
MAE: 18.62
MSE: 858.49
RMSE: 29.30
R2: 0.95
Adjusted R2: 0.95
***** Station 24 *****
MAE: 27.02
MSE: 1945.62
RMSE: 44.11
R2: 0.98
Adjusted R2: 0.98
***** Station 41 *****
MAE: 65.22
MSE: 12098.25
RMSE: 109.99
R2: 0.99
Adjusted R2: 0.99
***** Station 42 *****
MAE: 32.47
MSE: 2826.76
RMSE: 53.17
R2: 0.99
Adjusted R2: 0.99
***** Station 43 *****
MAE: 103.76
MSE: 34494.24
RMSE: 185.73
R2: 0.98
Adjusted R2: 0.98
***** Station 45 *****
MAE: 51.08
MSE: 8261.44
RMSE: 90.89
R2: 0.98
Adjusted R2: 0.98
***** Station 48 *****
MAE: 59.43
MSE: 8542.01
RMSE: 92.42
R2: 0.97
Adjusted R2: 0.97
***** Station 50 *****
MAE: 13.93
MSE: 383.42
RMSE: 19.58
R2: 0.95
Adjusted R2: 0.95
***** Station 68 *****
MAE: 39.96
MSE: 3510.48
RMSE: 59.25
R2: 0.97
Adjusted R2: 0.97
***** Station 74 *****
MAE: 53.27
MSE: 7741.10
RMSE: 87.9